# Production wire-scan data analysis
> 07.15.2021

In [ ]:
import sys
import importlib
from pprint import pprint

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import proplot as plot
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

sys.path.append('/Users/46h/Research/code/')
from accphys.tools import utils
from accphys.tools import beam_analysis as ba 
from accphys.tools import plotting as myplt
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled
from accphys.tools.accphys_utils import Sigma_from_twiss2D
from accphys.tools import emittance_measurement as em

In [ ]:
plot.rc['grid.alpha'] = 0.04
plot.rc['figure.facecolor'] = 'white'

## Load data 

### Settings 

In [ ]:
rec_node_id = 'Begin_Of_RTBT1'
# rec_node_id = 'Begin_Of_RTBT2'
# rec_node_id = 'RTBT_Diag:BPM19'
# rec_node_id = 'RTBT_Diag:WS20'
# rec_node_id = 'RTBT_Diag:BLM23'

In [ ]:
ws_ids = ['WS20', 'WS21', 'WS23', 'WS24']

In [ ]:
exp_twiss_filename = 'data/twiss/model_twiss_{}.dat'.format(rec_node_id)
transfer_matrices_filename = 'data/transfer_matrix/old/model_transfer_mat_elems_default_{}.dat'.format(rec_node_id)

Separate harp and wire-scanner files.

In [ ]:
def is_harp_file(filename):
    file = open(filename, 'r')
    for line in file:
        if 'Harp' in line:
            return True
    return False

filenames = utils.list_files('./data/ws/')
filenames_ws = [filename for filename in filenames if not is_harp_file(filename)]
filenames_harp = [filename for filename in filenames if is_harp_file(filename)]

print('Wire-scanner files')
pprint(filenames_ws)
print('')
print('Harp files')
pprint(filenames_ws)

### Harp 

These aren't used.

In [ ]:
harp_profiles = [em.read_pta_harp(filename) for filename in filenames_harp]

In [ ]:
fig, axes = plot.subplots(ncols=3, figsize=(8, 2), spanx=False)
axes[0].set_xlabel('x [mm]')
axes[1].set_xlabel('y [mm]')
axes[2].set_xlabel('u [mm]')
axes[0].set_ylabel('Amplitude')
for profile in harp_profiles:
    signals = [profile.hor, profile.ver, profile.dia]
    for ax, signal in zip(axes, signals):
        ax.plot(signal.pos, signal.raw, marker=None, lw=1, color='k')
axes.grid(axis='x')
axes.format(suptitle='Harp profiles')

### Wire-scanners 

In [ ]:
measurements = [em.read_pta_ws(filename) for filename in filenames_ws]

In [ ]:
axes = em.plot_profiles(measurements, ws_ids, fit=False, kws_raw=dict(marker='.', ms=3, lw=0))
plt.savefig('figures/profiles.png', dpi=300, facecolor='white')

In [ ]:
# kws_fit = dict(legend=False, color='k', alpha=0.2, zorder=0)
# axes = em.plot_profiles(measurements, ws_ids, fit=True, kws_raw=kws_raw, kws_fit=kws_fit)
# plt.savefig('figures/profiles_fit.png', dpi=300, facecolor='white')

## Reconstruct covariance matrix

Collect the measured beam moments at each wire-scanner.

In [ ]:
diag_wire_angle = np.radians(-45.0)

In [ ]:
moments = {ws_id: [] for ws_id in ws_ids}
for ws_id in ws_ids:
    for measurement in measurements:
        profile = measurement[ws_id]
        sig_xx = profile.hor.stats['Sigma'].rms**2
        sig_yy = profile.ver.stats['Sigma'].rms**2
        sig_uu = profile.dia.stats['Sigma'].rms**2
        sig_xy = em.get_sig_xy(sig_xx, sig_yy, sig_uu, diag_wire_angle)
        moments[ws_id].append([sig_xx, sig_yy, sig_xy])
    moments[ws_id] = np.array(moments[ws_id])

In [ ]:
fig, axes = plot.subplots(ncols=4, figsize=(9, 2))
plt_kws = dict(marker='.')
for ws_id, ax in zip(ws_ids, axes):
    ax.plot(moments[ws_id][:, 0], **plt_kws)
    ax.plot(moments[ws_id][:, 1], **plt_kws)
    ax.plot(moments[ws_id][:, 2], **plt_kws)
axes.format(ylabel=r'[mm$^2$]', xlabel='Measurement index', toplabels=ws_ids)
ax.legend(labels=[r'$\langle{x^2}\rangle$', r'$\langle{y^2}\rangle$', r'$\langle{xy}\rangle$'],
          ncols=1, loc=(1.02, 0), fontsize='small')
plt.savefig('figures/moments.png', dpi=300, facecolor='white')

Collect the linear transfer matrices at each wire-scanner.

In [ ]:
transfer_mats = {ws_id: [] for ws_id in ws_ids}
for ws_index, ws_id in enumerate(['WS02', 'WS20', 'WS21', 'WS23', 'WS24']):
    if ws_id == 'WS02': # don't have data from this wire-scanner
        continue
    for filename in filenames_ws:
        filename = 'data/transfer_matrix/model_transfer_mat_elems_default_Begin_Of_RTBT1_{}.dat'.format(filename.split('/')[-1]) 
        matrix_elements = np.loadtxt(filename)[ws_index]
        transfer_mats[ws_id].append(matrix_elements.reshape((4, 4)))

Form list from dictionaries.

In [ ]:
moments_list, transfer_mats_list = [], []
for ws_id in ws_ids:
    moments_list.extend(moments[ws_id])
    transfer_mats_list.extend(transfer_mats[ws_id])

Reconstruct the covariance matrix at the RTBT entrance.

In [ ]:
Sigma = em.reconstruct(transfer_mats_list, moments_list, lsq_solver='exact',
                       verbose=2, lsmr_tol=1e-5)

## Analysis 

In [ ]:
alpha_x, alpha_y, beta_x, beta_y, eps_x, eps_y = ba.get_twiss2D(Sigma)
eps_1, eps_2 = ba.intrinsic_emittances(Sigma)

print('Sigma:')
print(Sigma)
print('eps_4D = {:.3f}'.format(np.sqrt(np.linalg.det(Sigma))))
print('eps_1, eps_2 = {:.3f}, {:.3f}'.format(eps_1, eps_2))
print('eps_x, eps_y = {:.3f}, {:.3f}'.format(eps_x, eps_y))
print('alpha_x, alpha_y = {:.3f}, {:.3f}'.format(alpha_x, alpha_y))
print('beta_x, beta_y = {:.3f}, {:.3f}'.format(beta_x, beta_y))

In [ ]:
correlation_matrix = utils.cov2corr(Sigma)
print(correlation_matrix)

In [ ]:
coupling_coeff = 1.0 - np.sqrt((eps_1 * eps_2) / (eps_x * eps_y))
print('Coupling coefficient = {}'.format(coupling_coeff))

Below are the rms ellipses defined by the covariance matrix. The cross-plane projections are squares in reality.

In [ ]:
axes = myplt.rms_ellipses(Sigma, fill=True)
for i in range(3):
    for j in range(i + 1):
        kws = dict(lw=1, alpha=0.05, color='k', zorder=0)
        ax = axes[i, j]
        ax.axvline(0, **kws)
        ax.axhline(0, **kws)
        ax.grid(False)
plt.savefig('figures/rms_ellipses.png', dpi=300, facecolor='white');

Expected covariance matrix at RTBT entrance (from default optics).

In [ ]:
twiss_exp = np.loadtxt(exp_twiss_filename)
alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp = twiss_exp
print('alpha_x_exp, alpha_y_exp = {:.3f}, {:.3f}'.format(alpha_x_exp, alpha_y_exp))
print('beta_x_exp, beta_y_exp = {:.3f}, {:.3f}'.format(beta_x_exp, beta_y_exp))
Sigma_exp = Sigma_from_twiss2D(alpha_x_exp, alpha_y_exp, beta_x_exp, beta_y_exp, eps_x, eps_y)

In [ ]:
axes = em.plot_reconstructed_phasespace(transfer_mats, moments, Sigma, Sigma_exp)
axes.format(suptitle='Reconstructed phase space')
plt.savefig('figures/lines.png', dpi=300, facecolor='white')

In [ ]:
axes = em.plot_reconstructed_phasespace(transfer_mats, moments, 
                                        Sigma, Sigma_exp=None, 
                                        twiss=(alpha_x, alpha_y, beta_x, beta_y))
axes.format(suptitle='Reconstructed phase space (normalized by measured Twiss)')
plt.savefig('figures/lines_norm.png', dpi=300, facecolor='white')

### Comparison 

In [ ]:
def solve(lsq_solver):
    Sigma = em.reconstruct(transfer_mats_list, moments_list, verbose=0, lsq_solver=lsq_solver)
    eps_x, eps_y = ba.apparent_emittances(Sigma)
    eps_1, eps_2 = ba.intrinsic_emittances(Sigma)
    return eps_x, eps_y, eps_1, eps_2

def get_moments(n_meas):
    """Use `n_meas` measurements from each wire-scanner."""
    moments_list = []
    for measurement in measurements[:n_meas]:
        for ws_id in ws_ids:
            profile = measurement[ws_id]
            sig_xx = profile.hor.stats['Sigma'].rms**2
            sig_yy = profile.ver.stats['Sigma'].rms**2
            sig_uu = profile.dia.stats['Sigma'].rms**2
            sig_xy = em.get_sig_xy(sig_xx, sig_yy, sig_uu, diag_wire_angle)
            moments_list.append([sig_xx, sig_yy, sig_xy])
    return moments_list

def get_transfer_mats(n_meas):
    """Get transfer matrix for each measurement."""
    transfer_mats_list = []
    for filename in filenames_ws[:n_meas]:
        filename = 'data/transfer_matrix/model_transfer_mat_elems_default_Begin_Of_RTBT1_{}.dat'.format(filename.split('/')[-1]) 
        for elements, ws_id in zip(np.loadtxt(filename), ['WS02', 'WS20', 'WS21', 'WS23', 'WS24']):
            if ws_id != 'WS02': # don't have data from this wire-scanner
                transfer_mats_list.append(elements.reshape(4, 4))
    return transfer_mats_list

In [ ]:
results = dict()
for lsq_solver in ['exact', 'lsmr']:
    data = []
    for n_meas in range(1, len(filenames_ws)):
        moments_list = get_moments(n_meas)
        transfer_mats_list = get_transfer_mats(n_meas)
        data.append(solve(lsq_solver))
    results[lsq_solver] = np.array(data)

In [ ]:
fig, axes = plot.subplots(ncols=2, figsize=(6, 2))
for ax, key in zip(axes, ['exact', 'lsmr']):
    ax.plot(results[key], marker='.')
    ax.set_title('lsq_solver = ' + key)
axes.format(xlabel='Measurements used', ylabel='[mm mrad]', grid=False, 
            xticks=range(0, len(filenames_ws)), xtickminor=False)
axes[1].legend(labels=[r"$\varepsilon_{}$".format(s) for s in ['x', 'y', '1', '2']], 
               ncols=1, loc=(1.02, 0))
plt.savefig('figures/comparison.png', dpi=300, facecolor='white')